In [1]:
import pandas as pd
import numpy as np

# 가상의 데이터 100개 생성
num_samples = 100
data = {
    'jeonse_ratio': np.random.rand(num_samples) * 0.5 + 0.6, # 0.6 ~ 1.1 사이
    'debt_ratio': np.random.rand(num_samples) * 0.8,       # 0.0 ~ 0.8 사이
    'building_age': np.random.randint(0, 30, num_samples),
    'risk_label': np.random.randint(0, 2, num_samples)      # 0(안전) 또는 1(위험)
}
df = pd.DataFrame(data)

# 'dummy_data.csv' 라는 이름으로 파일 저장
df.to_csv('dummy_data.csv', index=False)

print("dummy_data.csv 파일 생성 완료!")
print(df.head())

dummy_data.csv 파일 생성 완료!
   jeonse_ratio  debt_ratio  building_age  risk_label
0      0.789029    0.472404            13           0
1      0.926436    0.228028             4           1
2      0.920140    0.209395             9           1
3      0.665217    0.214957            21           0
4      0.742825    0.757744             4           0


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

# 1. Dataset 설명서(클래스) 정의
class JeonseDataset(Dataset):

    # 1-1. __init__(self, csv_path): 초기 준비
    def __init__(self, csv_path):
        # Pandas로 CSV 파일 읽기
        df = pd.read_csv(csv_path)

        # 입력 feature와 정답 label 분리
        # .values를 붙여 NumPy 배열로 변환 (이후 텐서 변환이 빠름)
        self.features = df.drop('risk_label', axis=1).values
        self.labels = df['risk_label'].values

        print(f"데이터 로드 완료. 총 샘플 개수: {len(self.labels)}")

    # 1-2. __len__(self): 총 데이터 개수 반환
    def __len__(self):
        return len(self.labels)

    # 1-3. __getitem__(self, idx): idx번째 데이터 1개 반환
    def __getitem__(self, idx):
        # NumPy 배열에서 idx번째 데이터 추출
        feature = self.features[idx]
        label = self.labels[idx]

        # 데이터를 PyTorch 텐서로 변환 (중요!)
        # feature는 float32, label도 float32 (BCELoss를 위해)
        feature_tensor = torch.tensor(feature, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.float32)

        # label을 [1.0] 처럼 1차원 텐서로 만들어주면 좋음 (모델 출력과 모양 맞추기)
        return feature_tensor, label_tensor.view(1)

# --- 3. Dataset 및 DataLoader 사용해보기 ---

# 3-1. Dataset 객체(설명서) 생성
csv_file_path='dummy_data.csv'
dataset=JeonseDataset(csv_file_path)

# 3-2. DataLoader 객체(공급기) 생성
# 배치 크기를 16으로 설정
# shuffle=True로 데이터를 섞음
data_loader=DataLoader(dataset, batch_size=16, shuffle=True)

print("\n--- DataLoader가 데이터를 묶어주는 방식 확인 ---")

# 3-3. DataLoader가 어떻게 작동하는지 1 epoch만 돌려보기
# 'iter'로 감싸고 'next'로 하나만 뽑아볼 수 있습니다.
features_batch, labels_batch = next(iter(data_loader))

print(f"총 100개 데이터를 batch_size=16으로 묶었습니다.")
print(f"첫 번째 배치의 Feature 모양: {features_batch.shape}")
print(f"첫 번째 배치의 Label 모양: {labels_batch.shape}")
print("-" * 30)
print("첫 번째 배치의 Feature (앞 5개):")
print(features_batch[:5])
print("\n첫 번째 배치의 Label (앞 5개):")
print(labels_batch[:5])

# 100개 데이터 / 배치 16개 = 6.25 -> 총 7개의 배치가 나옴 (6개*16 + 1개*4)
print("\n--- 전체 배치 순회 ---")
for i, (features, labels) in enumerate(data_loader):
    print(f"Batch {i}: Features shape {features.shape}, Labels shape {labels.shape}")

데이터 로드 완료. 총 샘플 개수: 100

--- DataLoader가 데이터를 묶어주는 방식 확인 ---
총 100개 데이터를 batch_size=16으로 묶었습니다.
첫 번째 배치의 Feature 모양: torch.Size([16, 3])
첫 번째 배치의 Label 모양: torch.Size([16, 1])
------------------------------
첫 번째 배치의 Feature (앞 5개):
tensor([[ 0.6267,  0.3192,  4.0000],
        [ 0.6550,  0.5061, 14.0000],
        [ 0.7134,  0.4461, 20.0000],
        [ 0.9902,  0.6022, 22.0000],
        [ 0.8612,  0.0829, 14.0000]])

첫 번째 배치의 Label (앞 5개):
tensor([[1.],
        [0.],
        [1.],
        [1.],
        [1.]])

--- 전체 배치 순회 ---
Batch 0: Features shape torch.Size([16, 3]), Labels shape torch.Size([16, 1])
Batch 1: Features shape torch.Size([16, 3]), Labels shape torch.Size([16, 1])
Batch 2: Features shape torch.Size([16, 3]), Labels shape torch.Size([16, 1])
Batch 3: Features shape torch.Size([16, 3]), Labels shape torch.Size([16, 1])
Batch 4: Features shape torch.Size([16, 3]), Labels shape torch.Size([16, 1])
Batch 5: Features shape torch.Size([16, 3]), Labels shape torch.Size([16, 1])
B